<h1 align="center"> Python for the Humanities and Social Sciences <br> <em>Data Manipulation</em> </h1>

## Info
- Scott Bailey (CIDR), *scottbailey@stanford.edu*
- Javier de la Rosa (CIDR), *versae@stanford.edu*
- Ashley Jester (CIDR/SSDS), *ajester@stanford.edu*

## Goal
By the end of our workshop today, we hope you'll be able to load in data into a Pandas `DataFrame`, perform basic cleaning and analysis, and visualize relevant aspects of a dataset. We will work with a dataset of tweets collected during the release of the Apple Watch.

## Topics
- Pandas Series and DataFrame
- Loading data in, null and missing data
- Describing data
- Column manipulation
- String manipulation
- Split-Apply-Combine
- Plotting:
  - Basic charts (line, bar, pie)
  - Histograms
  - Scatter plots
  - Boxplots, violinplots

## Setup and packages we need in our environment
We'll be using Anaconda with Jupyter Notebooks for this workshop. For setting up both, please see the [setup](setup.ipynb).

For this workshop, we'll need an environment with the following packages:
- `matplotlib`
- `pandas`
- `requests`
- `seaborn`, available in the `conda-forge` channel

## Pandas

From Jake Vanderplas' book [**Python Data Science Handbook**](http://shop.oreilly.com/product/0636920034919.do) (from which some code excerpts are used in this workshop):

> Pandas is a newer package built on top of NumPy, and provides an efficient implementation of a `DataFrame`. `DataFrame`s are essentially multidimensional arrays with attached row and column labels, and often with heterogeneous types and/or missing data. As well as offering a convenient storage interface for labeled data, Pandas implements a number of powerful data operations familiar to users of both database frameworks and spreadsheet programs.

In [ ]:
import numpy as np  # np becomes the namespace of numpy
import pandas as pd
import requests

# Set some options
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 10)

There are three main data structures in Pandas: `Series`, `DataFrame`, and `Index`. Pandas has a very decent [documentation](http://pandas.pydata.org/pandas-docs/stable/), and using Jupyter, any method help can be shown by appending the a `?` to the end and running the cell.

In [ ]:
# For example
pd.isnull?

## Data I/O

Pandas provides a few methods to load in and out data in CSVs, Excel spreadsheets, HDF, or even JSON format.

For example, click in the next URL of a CSV file containing twitter data during the release of the Apple Watch: http://bit.ly/python_workshop_data

In [ ]:
# Pandas can fetch data from a URL
pd.read_csv("http://bit.ly/python_workshop_data")

Let's save the previous data to a local file.

In [ ]:
with open("twitter.csv", "wb") as file:
    file.write(requests.get("http://bit.ly/python_workshop_data").content)

In [ ]:
pd.read_csv("twitter.csv")

Let's reload the CSV but this time specifying a index column

In [ ]:
df = pd.read_csv("twitter.csv", index_col="created_at")
df

Now we can just save the clean data to any format supported by Pandas

In [ ]:
df.to_csv("twitter_indexed.csv", encoding="utf8")

## `DataFrame` and `Series`

A `DataFrame` is a two-dimensional array with both flexible row indices and flexible column names. It can be seen as a generalization of a two-dimensional NumPy array, or a specialization of a dictionary in which each column name maps to a `Series` of column data. A `Series` is a one-dimensional array of indexed data. It can be seem as a specialized dictionary or a generalized NumPy array.

A `DataFrame` is made up of `Series` in a similar way in which a table is made up of columns. The only restriction os that ach column must be of the same data type.

In [ ]:
df = pd.read_csv("twitter.csv")
df.columns

Accessing columns can be done using the dot notation, `df.column_name`, or the dictionary notation, `df["column_name"]`.

In [ ]:
df["urls"]

In [ ]:
df.urls

`DataFrame`s can be sliced to extract just a set of the columns you are interested in. We just pass in a list of the columns we need to the slice and get a `DataFrame` back.

In [ ]:
df[["urls", "text"]]

All `DataFrame`s are indexed. If an index is not explictly provided Pandas will asign one, givinh each row a consecutive number. `Series` and slices keep these indices, which makes possible further operations such as merging or columns manipulation.

`DataFrames` are designed to operate at the column level, not the row level. However, a subset of rows can be visualized easily using a slice like in any Python list.

In [ ]:
df[10:15]

In [ ]:
df.urls[10:15]

In [ ]:
df[["urls"]][10:15]

And you can even access individual rows and mix index and rows.

In [ ]:
df[["urls", "text"]].loc[2:5]  # for non numeric indices

In [ ]:
df[["urls", "text"]].iloc[2:5]  # for nummeric indices

In [ ]:
df.ix[2:5, ["urls", "text"]]  # for mixed indices and columns

<div style="font-size: 1em; margin: 1em 0 1em 0; border: 1px solid #86989B; background-color: #f7f7f7; padding: 0;">
<p style="margin: 0; padding: 0.1em 0 0.1em 0.5em; color: white; border-bottom: 1px solid #86989B; font-weight: bold; background-color: #AFC1C4;">
Activity
</p>
<p style="margin: 0.5em 1em 0.5em 1em; padding: 0;">
Given the `DataFrame` defined above, write an expression to extract a `DataFrame` with the columns `text`, `user_screen_name`, `user_name`, `user_lang`, and `hashtags`. Show only the first 5 rows of it.
<br/>
<!-- * **Hint**: You could ...* -->
</p>
</div>

In [ ]:
# Write here your solution

## Indexing and Expressions

Operations performed using a column or `Series` are broadcast to each of the elements contained.

In [ ]:
df["id"] * 2

In [ ]:
"@" + df["user_name"] + ": " + df["text"]

In [ ]:
df["id"] > 0

Which allows for a more advanced and useful indexing as you can pass in an expression to a `DataFrame` to select content.

In [ ]:
df[df["id"] > 575043732472528896]

Basically any expression that evaluates to a `Series` of `True` and `False` values and share the index can be used. And conditions can be put together using logical operators for "and", `&`, "or", `|`, and "not", `~`, making the filter more precise and expressive.

In [ ]:
df[(df["id"] > 575043732472528896) & (len(df["user_mentions"]) > 5)]

Some string operations are also available at the column level on the `.str` attribute of `Series`.

In [ ]:
df["urls"].str.split()

So the previous selection could also be written as:

In [ ]:
df[(df["id"] > 575043732472528896) & (df["user_mentions"].str.len() > 5)]

<div style="font-size: 1em; margin: 1em 0 1em 0; border: 1px solid #86989B; background-color: #f7f7f7; padding: 0;">
<p style="margin: 0; padding: 0.1em 0 0.1em 0.5em; color: white; border-bottom: 1px solid #86989B; font-weight: bold; background-color: #AFC1C4;">
Activity
</p>
<p style="margin: 0.5em 1em 0.5em 1em; padding: 0;">
Given the `states` `DataFrame` defined below, write an expression to calculate the population density of each state.
<br/>
* **Hint**: Population density is defined as the number of people per unit of area.*
</p>
</div>

In [ ]:
population_dict = {'California': 38332521, 'Texas': 26448193, 'New York': 19651127,
                   'Florida': 19552860, 'Illinois': 12882135}
area_dict = {'California': 423967, 'Texas': 695662, 'New York': 141297,
             'Florida': 170312, 'Illinois': 149995}  # these are in km²
states = pd.DataFrame({'population': population_dict, 'area': area_dict})
states

In [ ]:
# Write your code here

## Manipulation

The fundamental way of manipulating the contents of `DataFrame` columns is by using the `apply()` method, which allows to call a user defined function to each of the elements in the `Series`. Unlike the `.str` attribute, `apply()` is a general way of transforming values.

In [ ]:
def count_links(text):
    links = text.split(",")
    count = len(links)
    return count

df["urls"].apply(count_links)  # urls are separated by comma

However our naive `count_links` function does not know how to handle missing data. We could ignore those values by dropping the `NaN`, which is the Pandas way of saying missing data, or by cleaning our dataset on import time.

In [ ]:
df["urls"].dropna().apply(count_links)

Cleaning the data at the beginning, at import time, and for the whole `DataFrame` is usually a good idea, since makes operating with it more consistent and lesss prone to error.

This also avoids us the hassle to drop `NaN`'s everytime. In our case we will:
- Filter out some columns we are not interested in
- Specify and index for thr `DataFrame`
- Provide data types for some columns
- Parse dates as Python `datetime` for columns containing dates as strings
- Replace `NaN` values by empty strings in string columns

And then show the first 5, this time using the `head()` method.

In [ ]:
columns = [
    "created_at", "id",
    "text", "lang", "possibly_sensitive", "user_screen_name",
    "hashtags", "media", "symbols", "urls",
    "place", "country"]  # columns we want
index_column = "created_at"
column_types = {
    "id": int,
    "possibly_sensitive": bool,
    "lat": float,
    "lon": float,
}
fill_nans = {
    'country': '',
    'hashtags': '',
    'lang': '',
    'media': '',
    'place': '',
    'symbols': '',
    'text': '',
    'urls': '',
    'user_lang': '',
    'user_location': '',
    'user_name': '',
    'user_screen_name': ''
}
date_columns = ["created_at"]
df = pd.read_csv("twitter.csv",
    parse_dates=date_columns,
    index_col=index_column,
    usecols=columns,
    dtype=column_types).fillna(value=fill_nans)
df.head(5)

Now, our `count_links` should work just fine.

In [ ]:
df["urls"].apply(count_links)

And since the result of `appply()` is another `Series`, we can even create a new column with the it to enrich a `DataFrame`.

In [ ]:
df["urls_count"] = df["urls"].apply(count_links)
df[["urls", "urls_count"]]

If we now wanted to know the distribution or histogram of the number of links, we could use the `.value_counts()` method of `Series`.

In [ ]:
df["urls_count"].value_counts()

<div style="font-size: 1em; margin: 1em 0 1em 0; border: 1px solid #86989B; background-color: #f7f7f7; padding: 0;">
<p style="margin: 0; padding: 0.1em 0 0.1em 0.5em; color: white; border-bottom: 1px solid #86989B; font-weight: bold; background-color: #AFC1C4;">
Activity
</p>
<p style="margin: 0.5em 1em 0.5em 1em; padding: 0;">
Given the twitter `DataFrame`, add a new column `length` with the length ot the `text`, and show the tweets with exactly 140 characters.
<br/>
</p>
</div>

In [ ]:
# Write your code here
df["length"] = df["text"].apply(...)
df[...][["text"]]

`Series` also have some handy functions to compute basic statistics, like the sum or the mean. For example, given the new column created above, let's compute the average lenght of the tweets.

In [ ]:
df["length"].mean()

### Grouping data

But what about the most tweeted language? Or the most prolific user? For this kind of operations we need to use what is called the [Split-Apply-Combine](https://www.jstatsoft.org/article/view/v040i01/v40i01.pdf) approach:
- *Split* up a dataset
- *Apply* a function to each piece
- *Combine* all the pieces back together

<figure>
  <img src="https://swcarpentry.github.io/r-novice-gapminder/fig/splitapply.png" alt="Split-Apply-Combine">
  <figcaption>* Split-Apply-Combine - Source: [Software Carpentry](https://software-carpentry.org/lessons/). *</figcaption>
</figure>

In Pandas this can take the form of a `.groupby()` (split) operation followed by an `.aggregate()` (apply) function. Aggregates are like `apply()` that operate at the group level. Combining is done automatically for us by Pandas.

In [ ]:
df.groupby("lang")

In [ ]:
df.groupby("lang")[["text"]]  # no computation is made yet!

In [ ]:
def count_nonzero(items):
    total = 0
    for item in items:
        if item != 0:
            total += 1
    return total

df.groupby("lang")[["text"]].aggregate(count_nonzero)

`DataFrames` can be sorted by the values of one or more columns, in either ascending or descending order.

In [ ]:
aggregated = df.groupby("lang")[["text"]].aggregate(count_nonzero)
aggregated.sort_values("text", ascending=False)

However, for complex groupings like, creating a pivot table can be more useful.

In [ ]:
df.pivot_table(
    index=["lang", "user_screen_name"],
    values=["text"],
    aggfunc=count_nonzero
).sort_values("text", ascending=False)

<div style="font-size: 1em; margin: 1em 0 1em 0; border: 1px solid #86989B; background-color: #f7f7f7; padding: 0;">
<p style="margin: 0; padding: 0.1em 0 0.1em 0.5em; color: white; border-bottom: 1px solid #86989B; font-weight: bold; background-color: #AFC1C4;">
Activity
</p>
<p style="margin: 0.5em 1em 0.5em 1em; padding: 0;">
Given the twitter `DataFrame`, show the most popular retweet written in English.
<br/>
* **Hint**: In our dataset, retweets are tweets that start with "RT @".*
</p>
</div>

In [ ]:
# Write your code here

## Visualization

Pandas also provides some utilities to create basic plots just by calling `plot()` on a `Series` or `DataFrame`. But first we need to tell Jupyter that we are going to plot some charts using the plotting library matplotlib.

In [ ]:
# enables inline plotting in Jupyter using matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
df.groupby("lang")[["lang"]].aggregate(count_nonzero).plot()

Each time you call `plot()` an `Axes` object is returned, and Jupyter knows how to paint those. `Axes` objects are objects of the underlying `matplotlib` library for plotting in Python, and as such, lots of different options can be given to customize the aspect.

In [ ]:
ax = df.groupby("lang")[["lang"]].aggregate(count_nonzero).plot(
    kind="bar",
    figsize=(15, 5),
    title="# Tweets per Language",
    legend=None
)
ax.set_ylabel("Languagae")
ax.set_xlabel("# Tweets")

`Axes` can also be created empty using `matplotlib` and then put some content in them.

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 5))
df.groupby("lang")[["lang"]].aggregate(count_nonzero).plot(ax=ax,
    kind="bar",
    title="# Tweets per Language",
    legend=None
)
ax.set_ylabel("Languagae")
ax.set_xlabel("# Tweets")

There are other styles available as well.

In [ ]:
plt.style.available

In [ ]:
with plt.style.context('ggplot'):
    df.groupby("lang")[["lang"]].aggregate(count_nonzero).plot()

In [ ]:
# Even a special one for XKCD!
with plt.xkcd():
    df.groupby("lang")[["lang"]].aggregate(count_nonzero).plot()

`seaborn`, a convenience wrapper around `matplotlib`, changes the default style after being imported, but it can be reverted back easily setting the default style to `classic` using `plt.style.use("classic")`.

In [ ]:
import seaborn as sns
df.groupby("lang")[["lang"]].aggregate(count_nonzero).plot()

Let's create a hitogram with the lengths of tweets.

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 5))
df["length"].hist(ax=ax, bins=15, normed=True, color='lightseagreen')
df["length"].plot(ax=ax, kind='kde', xlim=(0, 150), style='r--')
ax.set_title("Histogram of lengths of tweets")

Boxplots are available by default.

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 6))
df.boxplot(column="length", grid=False, ax=ax)

Although violinplots can be used through `seaborn`.

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 6))
sns.violinplot(y=df["length"], grid=False, ax=ax)

<div style="font-size: 1em; margin: 1em 0 1em 0; border: 1px solid #86989B; background-color: #f7f7f7; padding: 0;">
<p style="margin: 0; padding: 0.1em 0 0.1em 0.5em; color: white; border-bottom: 1px solid #86989B; font-weight: bold; background-color: #AFC1C4;">
Activity
</p>
<p style="margin: 0.5em 1em 0.5em 1em; padding: 0;">
Given the twitter `DataFrame`, let's try to find out visually if there is any sort of relationship between the length of a tweet and the number of hastags it uses.
</p>
</div>

In [ ]:
# Write your code here
df["hashtags_count"] = 

fig, ax = plt.subplots(1, figsize=(15, 5))
ax.scatter(x=..., y=...)
ax.set_ylabel("Length")
ax.set_xlabel("# Hashtags")
ax.set_title("Tweet length by number of hashtags")